In [1]:
import os

import numpy as np
import pandas as pd
from pkg_resources import resource_filename

from flip import fisher, utils, data_vector
from flip.covariance import covariance

flip_base = resource_filename("flip", ".")
data_path = os.path.join(flip_base, "data")

/tmp/ipykernel_967235/3270887735.py:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename
[ 000000.00 ]: 09-18 12:01  root            INFO     No pypower module detected, gridding with this method is unavailable


In [2]:
sn_data = pd.read_parquet(os.path.join(data_path, "velocity_data.parquet"))

In [3]:
sn_data = sn_data[np.array(sn_data["status"]) != False]
sn_data = sn_data[np.array(sn_data["status"]) != None]

In [5]:
DataFisher = data_vector.FisherVelFromHDres(sn_data.to_dict(orient='list'))
print("Data free parameters: ", DataFisher.free_par)


Data free parameters:  ['sigma_M']


In [6]:
ktt, ptt = np.loadtxt(os.path.join(data_path, "power_spectrum_tt.txt"))
kmt, pmt = np.loadtxt(os.path.join(data_path, "power_spectrum_mt.txt"))
kmm, pmm = np.loadtxt(os.path.join(data_path, "power_spectrum_mm.txt"))

sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}

### Compute covariance
size_batch = 10_000
number_worker = 16

covariance_fit = DataFisher.compute_cov(
    "rcrk24",
    power_spectrum_dict,
    size_batch=size_batch,
    number_worker=number_worker,
    variant = 'growth_index'
)
print("Covariance free parameters: ", covariance_fit.free_par)


/pscratch/sd/b/bastienc/Soft/.miniconda3/envs/flip/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000000.86 ]: 09-18 12:01  root            INFO     Covariance matrix generated from flip with rcrk24 model in 1.29e+00 seconds


Covariance free parameters:  ['Om0', 'gamma', 'sigv']


In [7]:
fisher_properties = {
    "inversion_method": "inverse",
}


In [12]:
parameter_dict = {
    "gamma": 0.55,
    "Om0": 0.3,
    "sigv": 200,
    "sigma_M": 0.12
}

Fisher = fisher.FisherMatrix.init_from_covariance(
    covariance_fit,
    DataFisher,
    parameter_dict,
    fisher_properties=fisher_properties,
)

parameter_name_list, fisher_matrix = Fisher.compute_fisher_matrix()



In [14]:
fisher_matrix

array([[101.40216964, -60.04931404],
       [-60.04931404,  35.56547077]])